In [2]:
import pandas as pd

# Defina os nomes dos arquivos de entrada e saída
arquivo_entrada = 'bcell_table_export_1744050449.csv'
arquivo_saida = 'tabela_Bcell.csv'

# Leitura do CSV de entrada
df = pd.read_csv(arquivo_entrada)

# Filtra as linhas onde a coluna 'Epitope - Object Type' é igual a 'Linear peptide'
df_filtrado = df[(df['Epitope - Object Type'] == 'Linear peptide') & 
                 (df['Epitope - Modified residues'].isna() | (df['Epitope - Modified residues'] == ''))]

# Seleção das colunas desejadas
colunas_desejadas = ['Assay ID - IEDB IRI', 'Epitope - Name', 'Assay - Qualitative Measure']
df_saida = df_filtrado[colunas_desejadas]

# Escrita do novo CSV de saída (sem o índice)
df_saida.to_csv(arquivo_saida, index=False)

print("CSV de saída criado com sucesso!")

import pandas as pd

# Defina os nomes dos arquivos de entrada e saída
arquivo_entrada = '/scratch/pedro.bacelar/Projeto-HIV/IEDB/HIV/epitope_table_export_1744050423.csv'
arquivo_saida = 'tabela_epitopos.csv'

# Leitura do CSV de entrada
df = pd.read_csv(arquivo_entrada)

# Filtra as linhas onde a coluna 'Epitope - Object Type' é igual a 'Linear peptide'
df_filtrado = df[(df['Epitope - Object Type'] == 'Linear peptide') & 
                 (df['Epitope - Modified Residue(s)'].isna() | (df['Epitope - Modified Residue(s)'] == ''))]

# Seleção das colunas desejadas
colunas_desejadas = ["Epitope ID - IEDB IRI", 'Epitope - Name', 'Epitope - Source Organism']
df_saida = df_filtrado[colunas_desejadas]

# Escrita do novo CSV de saída (sem o índice)
df_saida.to_csv(arquivo_saida, index=False)

print("CSV de saída criado com sucesso!")

import pandas as pd

# Defina os caminhos dos arquivos CSV
caminho_1 = 'tabela_Bcell.csv'
#caminho_2 = '/scratch/pedro.bacelar/Projeto-HIV/novo/tabelas_filtradas/tabela_MHC.csv'
#caminho_3 = '/scratch/pedro.bacelar/Projeto-HIV/novo/tabelas_filtradas/tabela_Tcell.csv'

# Leitura dos arquivos CSV
df1 = pd.read_csv(caminho_1)
#df2 = pd.read_csv(caminho_2)
#df3 = pd.read_csv(caminho_3)

# Junta todos os DataFrames em um único
df_unido = pd.concat([df1], ignore_index=True)

# Defina o caminho do arquivo de saída
arquivo_saida = 'tabela_assays_Bcell.csv'

# Salva o DataFrame unido em um novo CSV
df_unido.to_csv(arquivo_saida, index=False)

print("CSV combinado criado com sucesso!")


CSV de saída criado com sucesso!
CSV de saída criado com sucesso!
CSV combinado criado com sucesso!


In [3]:
import pandas as pd

# Carregar arquivos
epitopes_df = pd.read_csv("tabela_epitopos.csv")
assay_results_df = pd.read_csv("tabela_assays_Bcell.csv")

# Classes positivas
positive_classes = ["Positive", "Positive-Intermediate", "Positive-High"]

# Criar um DataFrame para contar os resultados positivos por epítopo
positive_counts = assay_results_df[assay_results_df["Assay - Qualitative Measure"].isin(positive_classes)]
positive_counts = positive_counts.groupby("Epitope - Name").size().reset_index(name="positive_count")

# Criar um DataFrame para contar o total de resultados por epítopo
total_counts = assay_results_df.groupby("Epitope - Name").size().reset_index(name="total_count")

# Mesclar os contadores em um único DataFrame
counts_df = pd.merge(positive_counts, total_counts, on="Epitope - Name", how="right")
counts_df["positive_count"] = counts_df["positive_count"].fillna(0).astype(int)

# Criar a coluna Score no formato 'positivos/totais'
counts_df["Score"] = counts_df["positive_count"].astype(str) + "/" + counts_df["total_count"].astype(str)

# Mesclar os scores com o DataFrame de epítopos original
epitopes_df = pd.merge(epitopes_df, counts_df[["Epitope - Name", "Score"]], on="Epitope - Name", how="left")

# Preencher scores ausentes com "0/0"
epitopes_df["Score"] = epitopes_df["Score"].fillna("0/0")

# Salvar o resultado
epitopes_df.to_csv("scores_Bcell.csv", index=False)

print("Arquivo gerado: scores_Bcell.csv")

import pandas as pd

def filtrar_arquivo(input_path, output_path):
    # Carregar o arquivo CSV
    df = pd.read_csv(input_path)

    # Filtrar as linhas onde o comprimento do "Epitope - Name" está entre 9 e 12 caracteres
    df_filtered = df[df['Epitope - Name'].apply(lambda x: 0 <= len(x) <= 50)]

    # Remover as colunas "Epitope ID - IEDB IRI" e "Epitope - Source Organism"
    df_filtered = df_filtered.drop(columns=['Epitope ID - IEDB IRI', 'Epitope - Source Organism'])

    # Salvar o novo arquivo CSV
    df_filtered.to_csv(output_path, index=False)
    print(f"Arquivo filtrado e limpo salvo em: {output_path}")

# Exemplo de uso
input_path = 'scores_Bcell.csv'  # Substitua com o caminho do arquivo original
output_path = 'tabela_Final_Bcell.csv'  # Substitua com o caminho do novo arquivo
filtrar_arquivo(input_path, output_path)


import pandas as pd

def criar_arquivo_com_nova_coluna(path_arquivo):
    # Carregar o arquivo CSV original
    df = pd.read_csv(path_arquivo)
    
    # Função para determinar se é epitopo
    def determinar_eh_epitopo(score):
        # Verificar o primeiro dígito do score
        primeiro_digito = score.split('/')[0]
        return 0 if primeiro_digito == '0' else 1
    
    # Aplicar a função para criar a nova coluna
    df['eh epitopo'] = df['Score'].apply(determinar_eh_epitopo)
    
    # Salvar o novo CSV com a coluna 'eh epitopo'
    novo_path = path_arquivo.replace('.csv', '_com_nova_coluna.csv')
    df.to_csv(novo_path, index=False)
    print(f'Novo arquivo salvo em: {novo_path}')

# Exemplo de uso
# Caminho do arquivo CSV
path_arquivo = 'tabela_Final_Bcell.csv'
criar_arquivo_com_nova_coluna(path_arquivo)

import pandas as pd

def separar_epitopos(path_arquivo):
    # Carregando o arquivo CSV
    df = pd.read_csv(path_arquivo)

    # Filtrando os epitopos de acordo com a coluna 'eh epitopo'
    sim_df = df[df['eh epitopo'] == 1]
    nao_df = df[(df['eh epitopo'] == 0) & (df['Score'] != '0/0')]

    # Salvando os nomes dos epitopos nos arquivos de texto correspondentes
    sim_df['Epitope - Name'].to_csv('simB.txt', index=False, header=False)
    nao_df['Epitope - Name'].to_csv('naoB.txt', index=False, header=False)

# Coloque o caminho do seu arquivo CSV aqui
path_arquivo = 'tabela_Final_Bcell_com_nova_coluna.csv'

# Chamando a função para separar os epitopos
separar_epitopos(path_arquivo)



Arquivo gerado: scores_Bcell.csv
Arquivo filtrado e limpo salvo em: tabela_Final_Bcell.csv
Novo arquivo salvo em: tabela_Final_Bcell_com_nova_coluna.csv


In [4]:
import pandas as pd

# Supondo que você tenha um arquivo CSV chamado 'dataset.csv'
df = pd.read_csv('tabela_assays_Bcell.csv')

# Contar o número de valores únicos na coluna 'Epitope - Name'
unique_epitopes = df['Epitope - Name'].nunique()

print(f'Número de epítopos únicos: {unique_epitopes}')


Número de epítopos únicos: 123
